# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f57a82f2f70>
├── 'a' --> tensor([[ 0.2408,  0.3960,  2.2436],
│                   [ 0.3124,  1.3141, -0.7231]])
└── 'x' --> <FastTreeValue 0x7f57a82f29d0>
    └── 'c' --> tensor([[ 0.9380, -0.0145, -0.3961,  2.7245],
                        [ 0.6035, -0.6274, -0.5217, -0.6471],
                        [-0.4517,  0.7708,  0.4864, -0.8005]])

In [4]:
t.a

tensor([[ 0.2408,  0.3960,  2.2436],
        [ 0.3124,  1.3141, -0.7231]])

In [5]:
%timeit t.a

72.5 ns ± 1.75 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f57a82f2f70>
├── 'a' --> tensor([[ 0.5388, -0.3685, -0.2813],
│                   [ 0.3224, -0.2359,  1.0286]])
└── 'x' --> <FastTreeValue 0x7f57a82f29d0>
    └── 'c' --> tensor([[ 0.9380, -0.0145, -0.3961,  2.7245],
                        [ 0.6035, -0.6274, -0.5217, -0.6471],
                        [-0.4517,  0.7708,  0.4864, -0.8005]])

In [7]:
%timeit t.a = new_value

73.7 ns ± 2.62 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.2408,  0.3960,  2.2436],
               [ 0.3124,  1.3141, -0.7231]]),
    x: Batch(
           c: tensor([[ 0.9380, -0.0145, -0.3961,  2.7245],
                      [ 0.6035, -0.6274, -0.5217, -0.6471],
                      [-0.4517,  0.7708,  0.4864, -0.8005]]),
       ),
)

In [10]:
b.a

tensor([[ 0.2408,  0.3960,  2.2436],
        [ 0.3124,  1.3141, -0.7231]])

In [11]:
%timeit b.a

70.1 ns ± 1.23 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.9382, -0.8292, -0.5702],
               [-0.7136,  0.1011, -0.4145]]),
    x: Batch(
           c: tensor([[ 0.9380, -0.0145, -0.3961,  2.7245],
                      [ 0.6035, -0.6274, -0.5217, -0.6471],
                      [-0.4517,  0.7708,  0.4864, -0.8005]]),
       ),
)

In [13]:
%timeit b.a = new_value

593 ns ± 22.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

955 ns ± 21.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.3 µs ± 263 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

256 µs ± 19.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

247 µs ± 4.84 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f56ee4f6f70>
├── 'a' --> tensor([[[ 0.2408,  0.3960,  2.2436],
│                    [ 0.3124,  1.3141, -0.7231]],
│           
│                   [[ 0.2408,  0.3960,  2.2436],
│                    [ 0.3124,  1.3141, -0.7231]],
│           
│                   [[ 0.2408,  0.3960,  2.2436],
│                    [ 0.3124,  1.3141, -0.7231]],
│           
│                   [[ 0.2408,  0.3960,  2.2436],
│                    [ 0.3124,  1.3141, -0.7231]],
│           
│                   [[ 0.2408,  0.3960,  2.2436],
│                    [ 0.3124,  1.3141, -0.7231]],
│           
│                   [[ 0.2408,  0.3960,  2.2436],
│                    [ 0.3124,  1.3141, -0.7231]],
│           
│                   [[ 0.2408,  0.3960,  2.2436],
│                    [ 0.3124,  1.3141, -0.7231]],
│           
│                   [[ 0.2408,  0.3960,  2.2436],
│                    [ 0.3124,  1.3141, -0.7231]]])
└── 'x' --> <FastTreeValue 0x7f56ee4f6fa0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

45.2 µs ± 1.82 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f56ee4f6f10>
├── 'a' --> tensor([[ 0.2408,  0.3960,  2.2436],
│                   [ 0.3124,  1.3141, -0.7231],
│                   [ 0.2408,  0.3960,  2.2436],
│                   [ 0.3124,  1.3141, -0.7231],
│                   [ 0.2408,  0.3960,  2.2436],
│                   [ 0.3124,  1.3141, -0.7231],
│                   [ 0.2408,  0.3960,  2.2436],
│                   [ 0.3124,  1.3141, -0.7231],
│                   [ 0.2408,  0.3960,  2.2436],
│                   [ 0.3124,  1.3141, -0.7231],
│                   [ 0.2408,  0.3960,  2.2436],
│                   [ 0.3124,  1.3141, -0.7231],
│                   [ 0.2408,  0.3960,  2.2436],
│                   [ 0.3124,  1.3141, -0.7231],
│                   [ 0.2408,  0.3960,  2.2436],
│                   [ 0.3124,  1.3141, -0.7231]])
└── 'x' --> <FastTreeValue 0x7f57a42844c0>
    └── 'c' --> tensor([[ 0.9380, -0.0145, -0.3961,  2.7245],
                        [ 0.6035, -0.6274, -0.5217, -0.6471],
                 

In [23]:
%timeit t_cat(trees)

39.2 µs ± 711 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

73.5 µs ± 1.71 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.2408,  0.3960,  2.2436],
                [ 0.3124,  1.3141, -0.7231]],
       
               [[ 0.2408,  0.3960,  2.2436],
                [ 0.3124,  1.3141, -0.7231]],
       
               [[ 0.2408,  0.3960,  2.2436],
                [ 0.3124,  1.3141, -0.7231]],
       
               [[ 0.2408,  0.3960,  2.2436],
                [ 0.3124,  1.3141, -0.7231]],
       
               [[ 0.2408,  0.3960,  2.2436],
                [ 0.3124,  1.3141, -0.7231]],
       
               [[ 0.2408,  0.3960,  2.2436],
                [ 0.3124,  1.3141, -0.7231]],
       
               [[ 0.2408,  0.3960,  2.2436],
                [ 0.3124,  1.3141, -0.7231]],
       
               [[ 0.2408,  0.3960,  2.2436],
                [ 0.3124,  1.3141, -0.7231]]]),
    x: Batch(
           c: tensor([[[ 0.9380, -0.0145, -0.3961,  2.7245],
                       [ 0.6035, -0.6274, -0.5217, -0.6471],
                       [-0.4517,  0.7708,  0.4864, -0.8005]],
         

In [26]:
%timeit Batch.stack(batches)

101 µs ± 3.02 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.2408,  0.3960,  2.2436],
               [ 0.3124,  1.3141, -0.7231],
               [ 0.2408,  0.3960,  2.2436],
               [ 0.3124,  1.3141, -0.7231],
               [ 0.2408,  0.3960,  2.2436],
               [ 0.3124,  1.3141, -0.7231],
               [ 0.2408,  0.3960,  2.2436],
               [ 0.3124,  1.3141, -0.7231],
               [ 0.2408,  0.3960,  2.2436],
               [ 0.3124,  1.3141, -0.7231],
               [ 0.2408,  0.3960,  2.2436],
               [ 0.3124,  1.3141, -0.7231],
               [ 0.2408,  0.3960,  2.2436],
               [ 0.3124,  1.3141, -0.7231],
               [ 0.2408,  0.3960,  2.2436],
               [ 0.3124,  1.3141, -0.7231]]),
    x: Batch(
           c: tensor([[ 0.9380, -0.0145, -0.3961,  2.7245],
                      [ 0.6035, -0.6274, -0.5217, -0.6471],
                      [-0.4517,  0.7708,  0.4864, -0.8005],
                      [ 0.9380, -0.0145, -0.3961,  2.7245],
                      [ 0.6035, -

In [28]:
%timeit Batch.cat(batches)

176 µs ± 5.67 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

552 µs ± 10.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
